# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Principal Component Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd
pd.options.mode.chained_assignment = None  # default='warn'


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_500Cases_CH4_/')


NVarsRed     = 20

valid_perc   = 20.

FixedMinVal  = 1.e-14

n_ics        = 500


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

In [3]:
input_vars_        = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PC/CleanVars_ToRed.csv', header=None).to_numpy()[0,:]
try:
    input_vars_not = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PC/CleanVars_NotToRed.csv', header=None).to_numpy()[0,:]
except:
    input_vars_not = []
input_vars         = np.append(input_vars_, input_vars_not)

In [4]:
Vars = list(input_vars_not)
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0    = []
for Var in input_vars_not:
    Vars0.append(Var+'0')
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
InputVar = ['t'] + Vars0

In [5]:
Header           = Vars[0]
for var in Vars[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PC/Vars.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

Header           = Vars0[0]
for var in Vars0[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PC/Vars0.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

In [6]:
# FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
# Data         = pd.read_csv(FileName, header=0)
# SimIdxs      = Data.to_numpy(int)


# FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
# Data         = pd.read_csv(FileName)
# ICVecs       = Data.to_numpy()
# n_ics         = len(ICVecs)


# P0Vec        = []
# EqRatio0Vec  = []
# T0Vec        = []
# for iC in range(n_ics):
#     Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
#     P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
#     EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
#     T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
# P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
# EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
# T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(n_ics):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)


for iC in range(n_ics):
    try:
        FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        DataNew          = pd.read_csv(FileName, header=0)
        Nts              = len(DataNew)
        DataTemp         = DataNew.iloc[[0]][Vars] #DataTemp.iloc[1][Vars]
        DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
        DataTemp.reset_index(drop=True, inplace=True)
        DataNew[Vars0]   = DataTemp
        DataTemp['t']    = DataNew['t']
        DataNew['t']     = np.maximum(DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy(), FixedMinVal)
        if (iC==0):
            Data         = DataNew
        else:
            Data         = pd.concat([Data, DataNew], axis=0)
    except:
        pass
Data.reset_index(drop=True, inplace=True)

In [7]:
Data

,t,T,OH,NO,NO2,N2,PC_1,PC_2,PC_3,PC_4,...,PC0_11,PC0_12,PC0_13,PC0_14,PC0_15,PC0_16,PC0_17,PC0_18,PC0_19,PC0_20
0,1.000000e-14,1413.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.643497,0.645357,0.086985,-0.148028,-0.064497,...,-0.000361,-0.001018,-0.000884,-0.000756,-0.000005,-0.000120,0.000300,-0.000300,-0.000044,-0.000092
1,1.859514e-08,1412.999587,3.454512e-10,4.505861e-19,2.413815e-25,0.643497,0.645356,0.086985,-0.148028,-0.064497,...,-0.000361,-0.001018,-0.000884,-0.000756,-0.000005,-0.000120,0.000300,-0.000300,-0.000044,-0.000092
2,3.753606e-08,1412.999580,3.472373e-10,4.658945e-19,2.591442e-25,0.643497,0.645356,0.086985,-0.148028,-0.064497,...,-0.000361,-0.001018,-0.000884,-0.000756,-0.000005,-0.000120,0.000300,-0.000300,-0.000044,-0.000092
3,5.682919e-08,1412.999572,3.489746e-10,4.816316e-19,2.781695e-25,0.643497,0.645356,0.086985,-0.148028,-0.064497,...,-0.000361,-0.001018,-0.000884,-0.000756,-0.000005,-0.000120,0.000300,-0.000300,-0.000044,-0.000092
4,7.648108e-08,1412.999564,3.506650e-10,4.978064e-19,2.985436e-25,0.643497,0.645356,0.086985,-0.148028,-0.064497,...,-0.000361,-0.001018,-0.000884,-0.000756,-0.000005,-0.000120,0.000300,-0.000300,-0.000044,-0.000092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,9.255359e+01,1982.717314,1.171337e-05,8.187226e-07,2.143673e-13,0.652669,-0.564278,0.317353,0.058230,-0.065437,...,-0.000217,-0.000859,-0.000620,-0.000640,-0.000033,-0.000069,0.000338,-0.000297,-0.000045,-0.000070
499996,9.427464e+01,1982.717314,1.171337e-05,8.187204e-07,2.143667e-13,0.652669,-0.564278,0.317353,0.058230,-0.065437,...,-0.000217,-0.000859,-0.000620,-0.000640,-0.000033,-0.000069,0.000338,-0.000297,-0.000045,-0.000070
499997,9.602769e+01,1982.717314,1.171337e-05,8.187185e-07,2.143662e-13,0.652669,-0.564278,0.317353,0.058230,-0.065437,...,-0.000217,-0.000859,-0.000620,-0.000640,-0.000033,-0.000069,0.000338,-0.000297,-0.000045,-0.000070
499998,9.781334e+01,1982.717314,1.171337e-05,8.187171e-07,2.143658e-13,0.652669,-0.564278,0.317353,0.058230,-0.065437,...,-0.000217,-0.000859,-0.000620,-0.000640,-0.000033,-0.000069,0.000338,-0.000297,-0.000045,-0.000070


In [8]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/')
except:
    pass

In [9]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [10]:
data_id    = 'res'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput        = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = DataNoZero[['t'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [11]:
data_id = 'ics'

Data_ics                     = Data[Data['t'] == FixedMinVal]
n_points_ics                 = len(Data_ics)

idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

n_valid                      = len(valid_idx)
n_train                      = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput     = Data_ics[['t'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [12]:
# data_id = 'scs'

# Mask                         = [i for i in range(1,len(Data)-1) if Data.iloc[i-51]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]
# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput     = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [13]:
# data_id = 'fcs'

# Mask                         = [i for i in range(-1,len(Data)-1) if Data.iloc[i+1]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]

# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=43)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput       = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)